In [1]:
import findspark
findspark.init()

import pyspark
import random
from pathlib import Path
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.appName('logreg2').getOrCreate()

In [10]:
# import os
# os.chdir('D:\Python environments\Pyspark_env\environments\Data\Python-and-Spark-for-Big-Data-master\Spark_for_Machine_Learning\Logistic_Regression')

In [11]:
path = Path('D:\Python environments\Pyspark_env\environments\Data\Python-and-Spark-for-Big-Data-master\Spark_for_Machine_Learning\Logistic_Regression')

In [13]:
# data=spark.sql('select * from titanic.csv')
data=spark.read.csv(f'{path}/titanic.csv',inferSchema=True,header=True)

In [7]:
os.listdir()

['.ipynb_checkpoints',
 'customer_churn.csv',
 'Logistic_Regression_Consulting_Project.ipynb',
 'Logistic_Regression_Consulting_Project_SOLUTIONS.ipynb',
 'Logistic_Regression_Example.ipynb',
 'new_customers.csv',
 'sample_libsvm_data.txt',
 'titanic.csv',
 'Titanic_Log_Regression_Code_Along.ipynb']

In [14]:
data.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [15]:
data.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [22]:
data.count()

891

In [17]:
my_cols=data.select(['Survived','Pclass','Sex','Age','SibSp','Parch','Ticket','Fare','Embarked'])

In [18]:
final_data=my_cols.na.drop()

In [21]:
final_data.count()

712

In [39]:
!pip install py4j



You should consider upgrading via the 'd:\python environments\pyspark_env\environments\pyspark_env\scripts\python.exe -m pip install --upgrade pip' command.


In [41]:
!pip install --upgrade pip

ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'd:\\python environments\\pyspark_env\\environments\\pyspark_env\\scripts\\pip.exe'
Consider using the `--user` option or check the permissions.




  Attempting uninstall: pip
    Found existing installation: pip 20.1.1
    Uninstalling pip-20.1.1:


In [40]:
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,OneHotEncoder,StringIndexer)

gender_indexer=StringIndexer(inputCol='Sex',outputCol='SexIndex')
gender_encoder=OneHotEncoder(inputCol='SexIndex',outputCol='SexVec')

embark_indexer=StringIndexer(inputCol='Embark',outputCol='EmbarkIndex')
embark_encoder=OneHotEncoder(inputCol='EmbarkIndex',outputCol='EmbarkVec')

assembler=VectorAssembler(inputCols=['Pclass','SexVec','Age','SibSp','Parch','Ticket','Fare','EmbarkVec'],outputCol='features')

from pyspark.ml.classification import LogisticRegression

from pyspark.ml import Pipeline

log_reg_titanic=LogisticRegression(featuresCol='features',labelCol='Survived')

pipeline=Pipeline(stages=[gender_indexer,embark_indexer,
                         gender_encoder,embark_encoder,
                         assembler,log_reg_titanic])

train_data,test_data=final_data.randomSplit([0.7,0.3])

fitted_model=pipeline.fit(train_data)

Py4JJavaError: An error occurred while calling o223.fit.
: org.apache.spark.SparkException: Input column Embark does not exist.
	at org.apache.spark.ml.feature.StringIndexerBase.$anonfun$validateAndTransformSchema$2(StringIndexer.scala:128)
	at scala.collection.TraversableLike.$anonfun$flatMap$1(TraversableLike.scala:245)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.collection.IndexedSeqOptimized.foreach$(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:198)
	at scala.collection.TraversableLike.flatMap(TraversableLike.scala:245)
	at scala.collection.TraversableLike.flatMap$(TraversableLike.scala:242)
	at scala.collection.mutable.ArrayOps$ofRef.flatMap(ArrayOps.scala:198)
	at org.apache.spark.ml.feature.StringIndexerBase.validateAndTransformSchema(StringIndexer.scala:123)
	at org.apache.spark.ml.feature.StringIndexerBase.validateAndTransformSchema$(StringIndexer.scala:115)
	at org.apac